In [1]:
import os
import sys
import pprint
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import gfile
from tensorflow.compat.v1 import logging
import pprint
import _pickle as cPickle
import numpy as np

tf.disable_v2_behavior()

model_file = "./image_caption_data/checkpoint_inception_v3/inception_v3_graph_def.pb"
input_description_file = "./image_caption_data/results_20130124.token"
input_img_dir = "./image_caption_data/images/"
output_folder = "./image_caption_data/feature_extraction_inception_v3_bakup/"

batch_size = 1000

if not gfile.Exists(output_folder):
    gfile.MakeDirs(output_folder)

def parse_token_file(token_file):
    """Parses image description file."""
    img_name_to_tokens = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            img_id, description = line.strip('\r\n').split('\t')
            img_name, _ = img_id.split('#')
            img_name_to_tokens.setdefault(img_name, [])
            img_name_to_tokens[img_name].append(description)
    return img_name_to_tokens

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
img_name_to_tokens = parse_token_file(input_description_file)
all_img_names = img_name_to_tokens.keys()
logging.info("num of all images: %d" % len(all_img_names))
pprint.pprint(list(img_name_to_tokens.keys())[0:10])
pprint.pprint(img_name_to_tokens['1000092795.jpg'])

INFO:tensorflow:num of all images: 31783
['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg',
 '1000523639.jpg',
 '1000919630.jpg',
 '10010052.jpg',
 '1001465944.jpg',
 '1001545525.jpg']
['Two young guys with shaggy hair look at their hands while hanging out in the '
 'yard .',
 'Two young , White males are outside near many bushes .',
 'Two men in green shirts are standing in a yard .',
 'A man in a blue shirt standing in a garden .',
 'Two friends enjoy time spent together .']


In [3]:
def load_pretrained_inception_v3(model_file):
    with gfile.FastGFile(model_file, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name = "")

In [4]:
load_pretrained_inception_v3(model_file)

Instructions for updating:
Use tf.gfile.GFile.


In [8]:
num_batches = int(len(all_img_names) / batch_size)
if len(all_img_names) % batch_size != 0:
    num_batches += 1

with tf.Session() as sess:
    second_to_last_tensor = sess.graph.get_tensor_by_name("pool_3:0")
    for i in range(num_batches):
        batch_img_names = list(all_img_names)[i * batch_size: (i + 1) * batch_size]
        batch_features = []
        for img_name in batch_img_names:
            img_path = os.path.join(input_img_dir,img_name)
            logging.info("processing img %s" % img_name)
            if not gfile.Exists(img_path):
                raise Exception("%s doesn't exists" % img_path)
            img_data = gfile.FastGFile(img_path, "rb").read()
            feature_vector = sess.run(second_to_last_tensor,
                                      feed_dict = {
                                          "DecodeJpeg/contents:0" : img_data
                                      })
            batch_features.append(feature_vector)
        batch_features = np.vstack(batch_features)
        output_filename = os.path.join(output_folder, "image_features-%d.pickle" % i)
        logging.info("writing to file %s" % output_filename)
        with gfile.GFile(output_filename, 'w') as f:
            cPickle.dump((batch_img_names, batch_features), f)

INFO:tensorflow:processing img 1000092795.jpg
INFO:tensorflow:processing img 10002456.jpg
INFO:tensorflow:processing img 1000268201.jpg
INFO:tensorflow:processing img 1000344755.jpg
INFO:tensorflow:processing img 1000366164.jpg
INFO:tensorflow:processing img 1000523639.jpg
INFO:tensorflow:processing img 1000919630.jpg
INFO:tensorflow:processing img 10010052.jpg
INFO:tensorflow:processing img 1001465944.jpg
INFO:tensorflow:processing img 1001545525.jpg
INFO:tensorflow:processing img 1001573224.jpg
INFO:tensorflow:processing img 1001633352.jpg
INFO:tensorflow:processing img 1001773457.jpg
INFO:tensorflow:processing img 1001896054.jpg
INFO:tensorflow:processing img 100197432.jpg
INFO:tensorflow:processing img 100207720.jpg
INFO:tensorflow:processing img 1002674143.jpg
INFO:tensorflow:processing img 1003163366.jpg
INFO:tensorflow:processing img 1003420127.jpg
INFO:tensorflow:processing img 1003428081.jpg
INFO:tensorflow:processing img 100444898.jpg
INFO:tensorflow:processing img 1005216151

Exception: ./image_caption_data/images/4000008650.jpg doesn't exists